<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#VarianceComponentModels.jl" data-toc-modified-id="VarianceComponentModels.jl-1">VarianceComponentModels.jl</a></span><ul class="toc-item"><li><span><a href="#Package-Features" data-toc-modified-id="Package-Features-1.1">Package Features</a></span></li></ul></li><li><span><a href="#Heritability-Analysis" data-toc-modified-id="Heritability-Analysis-2">Heritability Analysis</a></span><ul class="toc-item"><li><span><a href="#Data-files" data-toc-modified-id="Data-files-2.1">Data files</a></span></li><li><span><a href="#Read-in-binary-SNP-data" data-toc-modified-id="Read-in-binary-SNP-data-2.2">Read in binary SNP data</a></span></li><li><span><a href="#EUR_subset" data-toc-modified-id="EUR_subset-2.3"><code>EUR_subset</code></a></span></li><li><span><a href="#Empirical-kinship-matrix" data-toc-modified-id="Empirical-kinship-matrix-2.4">Empirical kinship matrix</a></span></li><li><span><a href="#Simulating-phenotypes" data-toc-modified-id="Simulating-phenotypes-2.5">Simulating phenotypes</a></span></li><li><span><a href="#Phenotypes" data-toc-modified-id="Phenotypes-2.6">Phenotypes</a></span></li><li><span><a href="#Pre-processing-data-for-heritability-analysis" data-toc-modified-id="Pre-processing-data-for-heritability-analysis-2.7">Pre-processing data for heritability analysis</a></span></li><li><span><a href="#Heritability-of-single-trait" data-toc-modified-id="Heritability-of-single-trait-2.8">Heritability of single trait</a></span></li><li><span><a href="#Multivariate-trait-analysis" data-toc-modified-id="Multivariate-trait-analysis-2.9">Multivariate trait analysis</a></span></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-2.10">Exercise</a></span></li></ul></li><li><span><a href="#Testing-SNP-association-using-maximum-likelihoods-of-variance-component-models" data-toc-modified-id="Testing-SNP-association-using-maximum-likelihoods-of-variance-component-models-3">Testing SNP association using maximum likelihoods of variance component models</a></span><ul class="toc-item"><li><span><a href="#Fit-the-null-model" data-toc-modified-id="Fit-the-null-model-3.1">Fit the null model</a></span></li><li><span><a href="#Fit-the-alternative-model" data-toc-modified-id="Fit-the-alternative-model-3.2">Fit the alternative model</a></span></li><li><span><a href="#Likelihood-ratio-test" data-toc-modified-id="Likelihood-ratio-test-3.3">Likelihood ratio test</a></span></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-3.4">Exercise</a></span></li></ul></li></ul></div>

# Heritability analysis and testing SNP association using maximum likelihoods of variance component models 

**Lange Symposium**

**Juhyun Kim, juhkim111@ucla.edu**

**Department of Biostatistics, UCLA**

**Feb 22, 2020**

Machine information:

In [ ]:
versioninfo()

## VarianceComponentModels.jl

[`VarianceComponentModels.jl`](https://github.com/OpenMendel/VarianceComponentModels.jl/) is a package that resides in [OpenMendel](https://github.com/OpenMendel) ecosystem. It implements computation routines for fitting and testing variance component model of form 

$$\text{vec}(Y) \sim \text{Normal}(XB, \Sigma_1 \otimes V_1 + \cdots + \Sigma_m \otimes V_m)$$

where $\otimes$ is the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product). 

In this model, **data** is represented by 

* `Y`: `n x d` response matrix 
* `X`: `n x p` covariate matrix 
* `V=(V1,...,Vm)`: a tuple of `m` `n x n` covariance matrices

and **parameters** are

* `B`: `p x d` mean parameter matrix
* `Σ=(Σ1,...,Σm)`: a tuple of `m` `d x d` variance components. 



### Package Features 
* Maximum likelihood estimation (MLE) and restricted maximum likelihood estimation (REML) of mean parameters $B$ and variance component parameters $Σ$
* Allow constraints in the mean parameters $B$
* Choice of optimization algorithms: [Fisher scoring](https://books.google.com/books?id=QYqeYTftPNwC&lpg=PP1&pg=PA142#v=onepage&q&f=false) and [minorization-maximization algorithm](http://hua-zhou.github.io/media/pdf/ZhouHuZhouLange19VCMM.pdf)
* [Heritability analysis](https://openmendel.github.io/VarianceComponentModels.jl/latest/man/heritability/#Heritability-Analysis-1) in genetics

## Heritability Analysis

Variance component estimation can be used to estimate heritability of a quantitative trait. 

### Data files

For this analysis, we use a sample data set [`EUR_subset`](https://openmendel.github.io/SnpArrays.jl/latest/#Example-data-1) from `SnpArrays.jl`. This data set is available in the `data` folder of the package. 



In [ ]:
using SnpArrays

In [ ]:
datapath = normpath(SnpArrays.datadir())

`EUR_subset.bed`, `EUR_subset.bim`, and `EUR_subset.fam` is a set of Plink files in binary format.

In [ ]:
using Glob
readdir(glob"EUR_subset.*", datapath)

### Read in binary SNP data 

We use the [`SnpArrays.jl`](https://openmendel.github.io/SnpArrays.jl/latest) package to read in binary SNP data and compute the empirical kinship matrix. 

In [ ]:
# read in genotype data from Plink binary file
const EUR_subset = SnpArray(SnpArrays.datadir("EUR_subset.bed"))

### `EUR_subset` 

`EUR_subset` contains **379** individuals and **54,051** SNPs. There is no missing genotype in `EUR_subset`.

Minor allele frequencies (MAF) for each SNP:

In [ ]:
maf_EUR = maf(EUR_subset)

Histogram of minor allele frequency:

In [ ]:
## generate a histogram of MAF
# using Plots, PyPlot
# gr(size=(600, 500), html_output_format=:png)
# hist_maf = histogram(maf_EUR, xlab = "Minor Allele Frequency (MAF)", 
#                    ylab = "Number of SNPs", label="")
# png(hist_maf, "hist_MAF.png")

![image info](./hist_MAF.png)

Note that about 29% of SNPs are rare variants (MAF < 0.05). 

In [ ]:
count(!iszero, maf_EUR .< 0.05) / length(maf_EUR)

### Empirical kinship matrix

For a measure of relatedness, we compute empirical kinship matrix based on all SNPs by the genetic relation matrix (GRM). If there are missing genotypes, they are imputed on the fly by drawing according to the minor allele frequencies.

Kinship coefficients summarize genetic similarity between pairs of individuals. To estimate kinship coefficient $\Phi_{ij}$ between individuals $i$ and $j$ using GRM:

$$\widehat{\Phi}_{GRMij} = \frac{1}{2S} \sum_{k=1}^S \frac{(G_{ik}-2p_k)(G_{jk}-2p_k)}{2p_k(1-p_k)},$$

where 

* $S$: number of SNPs in this set
* $p_k$: minor allele frequency of SNP $k$
* $G_{ik} \in \{0,1,2\}$: number of copies of minor alleles at the $k$-th SNP of the $i$-th individual

In [ ]:
## GRM using SNPs with maf > 0.01 (default) 
Φgrm = grm(EUR_subset; method = :GRM) # classical genetic relationship matrix
# Φgrm = grm(EUR_subset; method = :MoM) # method of moment method
# Φgrm = grm(EUR_subset; method = :Robust) # robust method 

### Simulating phenotypes 


We simulate phenotype vector from

$$\mathbf{y} \sim \text{Normal}(\mathbf{1}, 0.1 \widehat{\Phi}_{GRM} + 0.9 \mathbf{I})$$

where $\widehat{\Phi}_{GRM}$ is the estimated empirical kinship matrix `Φgrm`. 

The data should be available in `pheno.txt`.

In [ ]:
## simulate `pheno.txt` 
# using LinearAlgebra, DelimitedFiles
# Random.seed!(1234)
# Ω = 0.1 * Φgrm + 0.9 * Matrix(1.0*I, nobs, nobs)
# Ωchol = cholesky(Symmetric(Ω))
# y = ones(nobs) + Ωchol.L * randn(nobs)
# writedlm("pheno.txt", y)

### Phenotypes 

Read in the phenotype data and plot a histogram.

In [ ]:
using DelimitedFiles 
pheno = readdlm("pheno.txt")

Histogram of phenotype values:

In [ ]:
## generate histogram of phenotype values
#hist_pheno = histogram(pheno, xlab="Phenotype", ylab="Frequency", label="")
#png(hist_pheno, "hist_pheno.png")

![image info](./hist_pheno.png)

### Pre-processing data for heritability analysis

To prepare variance component model fitting, we form an instance of VarianceComponentVariate. The two covariance matrices are $(2\Phi, I)$.

In [ ]:
using VarianceComponentModels, LinearAlgebra
# no. of observations 
nobs = size(pheno, 1)

# form data as VarianceComponentVariate
X = ones(nobs)
EURdata = VarianceComponentVariate(pheno, X, (2Φgrm, Matrix(1.0I, nobs, nobs)))
fieldnames(typeof(EURdata))

In [ ]:
EURdata

### Heritability of single trait 

Before fitting the variance component model, we pre-compute the eigen-decomposition of $2\Phi_{\text{GRM}}$, the rotated responses, and the constant part in log-likelihood, and store them as a TwoVarCompVariateRotate instance, which is re-used in various variane component estimation procedures.

We use Fisher scoring algorithm to fit variance component model for our trait. 

In [ ]:
# pre-compute eigen-decomposition 
@time EURdata_rotated = TwoVarCompVariateRotate(EURdata)
fieldnames(typeof(EURdata_rotated))

# form data set for trait 
trait_data = TwoVarCompVariateRotate(EURdata_rotated.Yrot, 
    EURdata_rotated.Xrot, EURdata_rotated.eigval, EURdata_rotated.eigvec, 
    EURdata_rotated.logdetV2)

# initialize model parameters
trait_model = VarianceComponentModel(trait_data)

# estimate variance components
_, _, _, Σcov, = mle_fs!(trait_model, trait_data; solver=:Ipopt, verbose=false)
σ2a = trait_model.Σ[1][1] # additive genetic variance 
σ2e = trait_model.Σ[2][1] # environmental variance 
@show σ2a, σ2e

Additive genetic variance:

In [ ]:
σ2a

Environmental/non-genetic variance:

In [ ]:
σ2e

In [ ]:
# heritability and its standard error from single trait analysis
h, hse = heritability(trait_model.Σ, Σcov)


[h[1], hse[1]]

We can also run MM algorithm. 

In [ ]:
trait_model = VarianceComponentModel(trait_data)
@time _, _, _, Σcov, = mle_mm!(trait_model, trait_data; verbose=false)
σ2a = trait_model.Σ[1][1]
σ2e = trait_model.Σ[2][1]
@show σ2a, σ2e

Heritability and its standard error.

In [ ]:
h, hse = heritability(trait_model.Σ, Σcov)
[h[1], hse[1]]

### Multivariate trait analysis

For the joint analysis of multiple traits, go to [`VarianceComponentModels` documentation](https://openmendel.github.io/VarianceComponentModels.jl/latest/man/heritability/). 

### Exercise

Repeat the above analysis computing the empirical kinship matrix using the method of moment method or the robust method. See [SnpArrays.jl documentation](https://openmendel.github.io/SnpArrays.jl/latest/#Genetic-relationship-matrix-1).

In [ ]:
# Φgrm = grm(EUR_subset; method = )

## Testing SNP association using maximum likelihoods of variance component models
credit: [Heritability tutorial by Sarah Ji, Janet Sinsheimer and Hua Zhou](https://github.com/OpenMendel/Tutorials/blob/master/Heritability/HERITABILITY-VCexample.ipynb)

Suppose we want to see a particular SNP has an effect on a given phenotype after accounting for relatedness among individuals. Here we fit variance component model with a single SNP *s* as fixed effect. 

$$\hspace{5em}  \mathbf{y} = \mathbf{X}\mathbf{\beta} + \mathbf{G}_s \gamma + \mathbf{g} + \mathbf{\epsilon} \hspace{5em} (1)$$

\begin{equation}
\begin{array}{ll}
\mathbf{g} \sim N(\mathbf{0}, \sigma_g^2\mathbf{\Phi}) \\
\mathbf{\epsilon} \sim N(\mathbf{0}, \sigma_e^2\mathbf{I})
\end{array}
\end{equation}

where 

* $\mathbf{y}$: phenotype 



and 


* Fixed effects:
    * $\mathbf{X}$: matrix of covariates including intercept
    * $\beta$: vector of covariate effects, including intercept
    * $\mathbf{G}_s$: genotype of SNP *s*
    * $\gamma$: (scalar) association parameter of interest, measuring the effect of genotype on phenotype  
* Random effects:
    * $\mathbf{g}$: random vector of polygenic effects with $\mathbf{g} \sim N(\mathbf{0}, \sigma_g^2 \mathbf{\Phi})$
        * $\sigma_g^2$: additive genetic variance
        * $\mathbf{\Phi}$: matrix of pairwise measures of genetic relatedness 
    * $\epsilon$: random vector with $\epsilon \sim N(\mathbf{0}, \sigma_e^2\mathbf{I})$
        * $\sigma_e^2$: non-genetic variance due to non-genetic effects assumed to be acting independently on individuals



To test whether SNP *s* is associated with phenotype, we fit two models. First consider the model without SNP *s* as fixed effects (aka null model): 

$$\hspace{5em}  \mathbf{y} = \mathbf{X}\mathbf{\beta} + \mathbf{g} + \mathbf{\epsilon} \hspace{5em} (2)$$

and the model with SNP *s* as fixed effects (1). Then we can compare the log likelihood to see if there is improvement in the model fit with inclusion of the SNP of interest. 

### Fit the null model

In [ ]:
using VarianceComponentModels
# null data model has two variance components but no SNP fixed effects
# form data as VarianceComponentVariate matrix 
X = ones(nobs)
nulldata = VarianceComponentVariate(pheno, X, (2Φgrm, Matrix(1.0I, nobs, nobs)))

In [ ]:
nullmodel = VarianceComponentModel(nulldata)

In [ ]:
@time nulllogl, nullmodel, = fit_mle!(nullmodel, nulldata; algo=:FS)

The null model log-likelihood (no SNP effects)

In [ ]:
nulllogl

The null model mean effects (a grand mean)

In [ ]:
nullmodel.B

The null model additive genetic variance

In [ ]:
nullmodel.Σ[1]

The null model environmental variance

In [ ]:
nullmodel.Σ[2]

### Fit the alternative model

In [ ]:
snp_vec = convert(Vector{Float64}, EUR_subset[:, 10]) 
Xalt = [ones(nobs) snp_vec]
altdata = VarianceComponentVariate(pheno, Xalt, (2Φgrm, Matrix(1.0I, nobs, nobs)))

In [ ]:
altmodel = VarianceComponentModel(altdata)

In [ ]:
@time altlogl, altmodel, = fit_mle!(altmodel, altdata; algo=:FS)

The alternative model log-likelihood:

In [ ]:
altlogl

The alternative model mean effects (a grand mean)

In [ ]:
altmodel.B

The alternative model additive genetic variance

In [ ]:
altmodel.Σ[1]

The alternative model environmental variance

In [ ]:
altmodel.Σ[2]

### Likelihood ratio test 

We use likelihood ratio test (LRT) to test the goodness-of-fit between two models. 

Our likelihood ratio test statistic is 1.786 (distributed chi-squared), with one degree of freedom.

In [ ]:
using Distributions
LRT = 2(altlogl - nulllogl)

The associated p-value: 

In [ ]:
pval = ccdf(Chisq(1), LRT)

We see that adding this SNP as a covariate to the model does not fit significantly better than the null model. In other words, the SNP does not explain more of the variation in our trait.

### Exercise

Use minorization-maximization algorithm (`algo=:MM`) to find MLEs of both null model and alternative model. Then conduct the likelihood ratio test.